In [1]:
import pandas as pd
import numpy as np
import os
import math
import yaml
from dateutil.relativedelta import relativedelta
from IPython.display import display
import yaml

#### Helpers ####
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import db_utils
from helpers import settings

bucket = S3BucketUtils()
################

def get_last_w_months_average(df, col_name, last_w_months_avg_col_name, w):
    df[last_w_months_avg_col_name] = df.groupby('spot_id')[col_name].\
    apply(lambda x: x.rolling(window=w, min_periods=0).mean())
    
    return df

def get_last_w_months_sum(df, col_name, last_w_months_avg_col_name, w):
    df[last_w_months_avg_col_name] = df.groupby('spot_id')[col_name].\
    apply(lambda x: x.rolling(window=w, min_periods=0).sum())
    
    return df

def get_num_of_months(df, col_name, num_of_months_col_name):
    df = df.merge(df[df[col_name]>0].groupby('spot_id')['time'].count().reset_index().\
rename(columns = {'time':num_of_months_col_name}), on = 'spot_id', how = 'left')
    df[num_of_months_col_name].fillna(0, inplace = True)
    
    return df

In [2]:
def prepare_data_for_calculating_stats(df, all_vars_new_names, var_, start_date, end_date):
    df_tmp = df.copy()
    var_ = all_vars_new_names[var_]
    df_tmp = df_tmp.merge(df_tmp[df_tmp[var_]>0].groupby('spot_id')['left_limit'].min().\
        reset_index().rename(columns = {'left_limit':var_+'_start'}), on = ['spot_id'])

    df_tmp = df_tmp.merge(df_tmp[df_tmp[var_]>0].groupby('spot_id')['left_limit'].max().\
    reset_index().rename(columns = {'left_limit':var_+'_end'}), on = ['spot_id'])

    df_tmp.drop(df_tmp[(df_tmp['left_limit']<df_tmp[var_+'_start'])|\
                      (df_tmp['left_limit']>df_tmp[var_+'_end'])].index, inplace = True)
    
    ## spots which had inquiries before the covid period ##
    spots_of_interest = df_tmp[(df_tmp[var_]>0)&\
                              (df_tmp['left_limit']<start_date)]['spot_id'].unique()
    
    df_tmp.drop(df_tmp[(df_tmp['left_limit']<start_date)|\
                      (df_tmp['left_limit']>end_date)|\
                      (df_tmp['spot_id'].isin(spots_of_interest)==False)].index, inplace = True)
    
    df_tmp.reset_index(drop = True, inplace = True)
    df_tmp = df_tmp.merge(df_tmp.groupby('spot_id')['time'].\
                          count().reset_index().rename(columns = {'time':var_+'_total_num_of_months'}),\
                          on = ['spot_id'])
    
    return (spots_of_interest, df_tmp)

In [3]:
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = model_params['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')

In [4]:
data_sets = ['data_tv_ALL_spots_with_CB_wo_151617.csv',\
             'data_tv_ALL_spots_wo_CB_wo_151617.csv',\
             'data_tv_CAN_CANCEL_spots_with_CB_wo_151617.csv',\
             'data_tv_CAN_CANCEL_spots_wo_CB_wo_151617.csv']

filename = data_sets[0].split('_wo_151617.csv')[0]

## ALL spots with CB ##
df = \
bucket.load_csv_from_s3(file_name='churn_analysis/data/'+date_dir+'/'+filename+'_wo_151617.csv')

In [5]:
vars_of_interest = ['private.parties.submissions.total',\
                        #'PP.Changed.inquiry.status.total',\
                       'reservations.submissions.total',\
                        #'R.Changed.inquiry.status.total'
                       ]

all_vars = vars_of_interest

In [6]:
all_vars_new_names = dict.fromkeys(all_vars)
for var_ in all_vars:
    var_new_name = var_
    if var_ == 'QR.code.flyer.scans.total':
        var_new_name = 'other_non_contactless_menu_qr_flyer_scans'
    elif var_ == 'Consumer.job.listings.inquiry.total':
        var_new_name = 'consumer_job_listing_inquiries'
    elif 'total' in var_:
        var_new_name = var_[0:-6]
        
    if '.' in var_:
        var_new_name = var_new_name.replace('.', '_')
    
    var_new_name = var_new_name.lower()
    
    all_vars_new_names[var_] = var_new_name

In [7]:
all_vars_new_names

{'private.parties.submissions.total': 'private_parties_submissions',
 'reservations.submissions.total': 'reservations_submissions'}

In [8]:
##############################################################################

In [9]:
## read covid period limits ##
with open(r'./parameters/covid_period.yaml') as file:
    covid_period_limits = yaml.load(file, Loader=yaml.FullLoader)

In [10]:
for var_ in all_vars:
    var_new_name = var_
    if var_ == 'QR.code.flyer.scans.total':
        var_new_name = 'other_non_contactless_menu_qr_flyer_scans'
    elif var_ == 'Consumer.job.listings.inquiry.total':
        var_new_name = 'consumer_job_listing_inquiries'
    elif 'total' in var_:
        var_new_name = var_[0:-6]
        
    if '.' in var_:
        var_new_name = var_new_name.replace('.', '_')
    
    var_new_name = var_new_name.lower()
    
    all_vars_new_names[var_] = var_new_name
    
    covid_period_lengths = []
    for i in range(1, int(len(covid_period_limits[all_vars_new_names[var_]])/2)+1):
        start_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_]]['start_'+str(i)])
        end_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_]]['end_'+str(i)])
        months_diff = relativedelta(end_date, start_date).months + 1
        covid_period_lengths.append(months_diff)
    
    
    for w in range(1, max(covid_period_lengths)+1):
        if w == 1:
            if all_vars_new_names[var_]+'_last_'+str(w)+'_month_sum' in df.columns:
                df.drop(all_vars_new_names[var_]+'_last_'+str(w)+'_month_sum', axis = 1, inplace = True)
            df = get_last_w_months_sum(df=df, col_name=var_, \
                                         last_w_months_avg_col_name=all_vars_new_names[var_]+'_last_'+str(w)+'_month_sum',\
                                         w = w)
        elif all_vars_new_names[var_]+'_last_'+str(w)+'_months_sum' in df.columns:
            df.drop(all_vars_new_names[var_]+'_last_'+str(w)+'_months_sum', axis = 1, inplace = True)
        df = get_last_w_months_sum(df=df, col_name=var_, \
                                         last_w_months_avg_col_name=all_vars_new_names[var_]+'_last_'+str(w)+'_months_sum',\
                                         w = w)
    first_approach_dir = 'data/'+date_dir+'/covid_period_analysis/first_approach/'+all_vars_new_names[var_]+'/'
    second_approach_dir = 'data/'+date_dir+'/covid_period_analysis/second_approach/'+all_vars_new_names[var_]+'/'
    df.rename(columns = {var_:all_vars_new_names[var_]}, inplace = True)
    
    var_orig = var_
    for approach in [1, 2]:
        for i in range(0, len(covid_period_lengths)):
            period_length = covid_period_lengths[i]
            start_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_orig]]['start_'+str(i+1)])
            end_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_orig]]['end_'+str(i+1)])
            if approach == 1:
                period_to_look_at = pd.read_csv(first_approach_dir+'period_'+str(i+1)+'.csv')
            elif approach == 2:
                period_to_look_at = pd.read_csv(second_approach_dir+'period_'+str(i+1)+'.csv')
                
            export = pd.DataFrame(columns = ['variable_name', 'mean', 'min', '10%',\
                                 '20%', '25%', '50%','75%', 'max'])
            export.loc[0, 'variable_name'] = all_vars_new_names[var_orig]
            export = export.merge(period_to_look_at[['variable_name', 'period_to_look_at']], on = ['variable_name'])
            export.set_index('variable_name', inplace = True)
            
            (spots_of_interest, df_tmp) = \
            prepare_data_for_calculating_stats(df=df, all_vars_new_names=all_vars_new_names,\
                                   var_=var_orig, start_date=str(start_date)[0:10], end_date=str(end_date)[0:10])
            
            var_ = all_vars_new_names[var_orig]
            period_to_look_at_v = \
            period_to_look_at[(period_to_look_at['variable_name']==var_)]['period_to_look_at'].values[0]
            
            if period_to_look_at_v == 'last_month':
                stats = df_tmp.groupby('spot_id')[var_].mean().\
                reset_index()[var_].describe()
                avg_values_per_spot = df_tmp.groupby('spot_id')[var_].mean().reset_index()
                avg_values_per_spot.sort_values(var_, ascending = True, inplace = True)
                export.loc[var_, '10%'] = \
                round(avg_values_per_spot[0:round(0.1*len(avg_values_per_spot))][var_].max(), 2)
                export.loc[var_, '20%'] = \
                round(avg_values_per_spot[0:round(0.2*len(avg_values_per_spot))][var_].max(), 2)
            else:
                stats = df_tmp.groupby('spot_id')[var_+'_'+period_to_look_at_v+'_sum'].mean().\
                reset_index()[var_+'_'+period_to_look_at_v+'_sum'].describe()
                avg_values_per_spot = df_tmp.groupby('spot_id')[var_+'_'+period_to_look_at_v+'_sum'].mean().reset_index()
                avg_values_per_spot.sort_values(var_+'_'+period_to_look_at_v+'_sum', ascending = True, inplace = True)
                export.loc[var_, '10%'] = \
                round(avg_values_per_spot[0:round(0.1*len(avg_values_per_spot))][var_+'_'+period_to_look_at_v+'_sum'].max(), 2)
                export.loc[var_, '20%'] = \
                round(avg_values_per_spot[0:round(0.2*len(avg_values_per_spot))][var_+'_'+period_to_look_at_v+'_sum'].max(), 2)

            export.loc[var_, 'mean'] = round(stats['mean'], 2)
            export.loc[var_, 'min'] = round(stats['min'], 2)
            export.loc[var_, '25%'] = round(stats['25%'], 2)
            export.loc[var_, '50%'] = round(stats['50%'], 2)
            export.loc[var_, '75%'] = round(stats['75%'], 2)
            export.loc[var_, 'max'] = round(stats['max'], 2)
            
            if approach == 1:
                export.\
                to_csv(first_approach_dir+'period_'+str(i+1)+'_stats.csv')
            elif approach == 2:
                export.\
                to_csv(second_approach_dir+'period_'+str(i+1)+'_stats.csv')

In [11]:
##### differences in first and second approach ######

In [12]:
for var_ in all_vars:
    first_approach_dir = 'data/'+date_dir+'/covid_period_analysis/first_approach/'+all_vars_new_names[var_]+'/'
    second_approach_dir = 'data/'+date_dir+'/covid_period_analysis/second_approach/'+all_vars_new_names[var_]+'/'
    covid_period_lengths = []
    for i in range(1, int(len(covid_period_limits[all_vars_new_names[var_]])/2)+1):
        start_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_]]['start_'+str(i)])
        end_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_]]['end_'+str(i)])
        months_diff = relativedelta(end_date, start_date).months + 1
        covid_period_lengths.append(months_diff)
        
    for i in range(0, len(covid_period_lengths)):
        period_length = covid_period_lengths[i]
        start_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_orig]]['start_'+str(i+1)])
        end_date = pd.to_datetime(covid_period_limits[all_vars_new_names[var_orig]]['end_'+str(i+1)]) 
        
        first_approach = \
        pd.read_csv(first_approach_dir+'period_'+str(i+1)+'_stats.csv')
        second_approach = \
        pd.read_csv(second_approach_dir+'period_'+str(i+1)+'_stats.csv')
        
        for col in first_approach.columns[1:]:
            first_approach.rename(columns = {col:col+'_first_approach'}, inplace = True)
        for col in second_approach.columns[1:]:
            second_approach.rename(columns = {col:col+'_second_approach'}, inplace = True)
            
        comparison = first_approach.merge(second_approach, on = ['variable_name'])
        print(var_)
        print('period_'+str(i+1))
        display(comparison[comparison['period_to_look_at_first_approach']!=comparison['period_to_look_at_second_approach']])
#         display(comparison)

private.parties.submissions.total
period_1


,variable_name,mean_first_approach,min_first_approach,10%_first_approach,20%_first_approach,25%_first_approach,50%_first_approach,75%_first_approach,max_first_approach,period_to_look_at_first_approach,mean_second_approach,min_second_approach,10%_second_approach,20%_second_approach,25%_second_approach,50%_second_approach,75%_second_approach,max_second_approach,period_to_look_at_second_approach
0,private_parties_submissions,26.02,0.0,1.58,3.4,4.62,12.56,33.69,446.75,last_7_months,17.37,0.0,1.0,2.17,3.02,7.75,20.31,321.0,last_5_months


reservations.submissions.total
period_1


,variable_name,mean_first_approach,min_first_approach,10%_first_approach,20%_first_approach,25%_first_approach,50%_first_approach,75%_first_approach,max_first_approach,period_to_look_at_first_approach,mean_second_approach,min_second_approach,10%_second_approach,20%_second_approach,25%_second_approach,50%_second_approach,75%_second_approach,max_second_approach,period_to_look_at_second_approach


reservations.submissions.total
period_2


,variable_name,mean_first_approach,min_first_approach,10%_first_approach,20%_first_approach,25%_first_approach,50%_first_approach,75%_first_approach,max_first_approach,period_to_look_at_first_approach,mean_second_approach,min_second_approach,10%_second_approach,20%_second_approach,25%_second_approach,50%_second_approach,75%_second_approach,max_second_approach,period_to_look_at_second_approach
